# Homework 5 - Visit the Wikipedia hyperlinks graph!

## Our team (group 29):

- Elena lavinia Diaconu
- Giulia Scikibu Maravalli
- Hassan Ismail

*Libraries*:

In [1]:
#import libraries:

import numpy as np
from collections import defaultdict
import pandas as pd
import re
import networkx as nx
import functions as func

## RQ1

Our first assignment is to built a graph where the nodes $(V)$ are wikipedia ariticles and the edges $(E)$ are the hyperlink among them, i.e. $G=(V,E)$.    
In order to make this graph, we based on [wiki-topcats-reduced.txt](https://drive.google.com/file/d/1ghPJ4g6XMCUDFQ2JPqAVveLyytG8gBfL/view). Taking information from this file, we create a dictionary, called edges, that has as key an article and as value a list of the other articles towards which the key article is connected. At the same time, we also create a set (nodes), where we collect all the unique articles (i.e. nodes) that appears in our graph.

In [2]:
f = open('wiki-topcats-reduced.txt')
edges = defaultdict(list)
nodes = set()

for line in f:
    (key, val) = line.split()
    
    #dictionary edges has as key the wiki page, as value a list of pages linked to the key page
    edges[int(key)].append(int(val))
    
    #add key and value to nodes (both key and values are wiki pages)
    #nodes is a set so if a page is already present it will not be added again
    nodes.add(int(key))
    nodes.add(int(val))

edges = dict(edges) #edges from default dictionary to dictionary

We used the information from *edges* dicitonary and *nodes* set to provide same basic information about the graph:  

- If it is direct or not
- The number of nodes
- The number of edges
- The average node degree. Is the graph dense?

**Directed or not?**  

First of all we want to check if the graph is directed or undirected. A graph is undirected if the edges have no direction (i.e. there two-way relationship between linked nodes), in simple terms if node A is connected to node B, also node B is connected to node A, and it applies to all the nodes in the graph. Therefore, if we know that just a couple of nodes are not mutually connected (e.g. A -> B, but B not connected to A), we can tell that the graph is directed.  
In our code we use the *edges* dictionary (reminder: key is a node and the value is a list of nodes towards which the key node is connected), we itererate through nodes in keys and values to see if they are mutually conncected, if not or if there is a KeyError (i.e. that node not exist as a key, so it is not linked toward anything) we know that the graph is directed and we print it, if nothing is printed out after iteration the graph is uniderected.  
In our case the graph is directed.

In [3]:
#iterate through nodes
for key, val in edges.items():
    try:
        for node in val:
            #if the key node is not in the list of nodes toward its linked node is connected
            #it means they are not mutually connected, then the graph is directed.
            if key not in edges[node]: 
                break
                print('Directed Graph')
    #if a KeyError is raised, that node not exist as a key, so it is not linked toward anything
    except KeyError:
        print('Directed Graph')
        break

Directed Graph


**Number of nodes**  

We compute the total number of nodes of our graph by taking the length of *nodes* set. This set, created in the first chunck of this script, is made of all the nodes that appear in *wiki-topcats-reduced.txt*.

In [4]:
#The number of nodes
print('The number of nodes:', len(nodes))

The number of nodes: 461193


**Number of edges and average node degree**    

Now taking information from edges dictionary we create another dictionary *degree*, where we have as key the page (as in edges dict) and as value the degree associated to that page (we assume that every edge has $degree = 1$, in each iteration we also sum the degrees, so that at the end of the for loop, we get in the variable *sum_degree* the sum of all node degrees (since every degree is equal to 1, the sum of all degree correspond to the sum of all the edges).

In [5]:
degree = {}
sum_degrees = 0
for key, value in edges.items():
    degree[key]=len(value)
    sum_degrees += degree[key]

#The number of edges
tot_edges = sum_degrees
print('Number of edges:', tot_edges)

#The average node degree
average_degree = sum_degrees/len(nodes)
print('The average node degree:', average_degree)

Number of edges: 2645247
The average node degree: 5.735661642739591


**Is the graph dense?**  

A graph is *dense* when its number of edges is close to the maximal number of edges. The graph density of a directed graph is defined as:
$$D=\frac{|E|}{|V||(V-1)|}$$  
Graph desity is a value between 0 and 1, where 1 is the maximal density and 0 is minimal density.

In [6]:
#maximal n. of edges:
max_edgs_n = (len(nodes)*(len(nodes)-1))
print('Maximal number of edges', max_edgs_n)

#graph density
density = tot_edges/max_edgs_n
print('Graph density is ', density)

Maximal number of edges 212698522056
Graph density is  1.2436602635647606e-05


As shown above the graph density is about 0.0000124, in light of it, we can claim that **the graph is not dense**.

## RQ2

### Ranking of categories

Now we have to rank all categories of our articles according to the distance from an input category, the distance is computed as follow:  
$$distance(C_{0}, C_{1}) = median(ShortestPath(C_{0}, C_{1}))$$

At first we need to load data from [wiki-topcats-categories.txt](https://snap.stanford.edu/data/wiki-topcats.html), in this file we have the name of the category with its corresponding articles, relying on this file we create a dictionary, named *categories*, in which the keys are the names of the categories and the values are lists of the corresponding articles.  
We keep just the categories with more than 3500 articles, then we remove from the remaining categories also all the nodes that are isolated (i.e. no edges incoming, no edges outgoing), in fact they do not provide useful information to our analysis since thay do not have edges. However we record how many isolated nodes we have removed from any category in isolated_nod dictionary (key is name of category, value the number of isolated nodes). Later, we will consider them as nodes with infinite distances and add them directly in the shortestpath list. 

We saved this dictionary in a file (*categories.txt*), that we include in our repositiory, feel free to take a look at it.

In [7]:
f = open('wiki-topcats-categories.txt')
categories = {}
isolated_nod = {}

for line in f.readlines():
    list_of_art = [int(s) for s in line.split() if s.isdigit()]
    #keep just categories with more than 3500 articles
    if len(list_of_art) > 3500:
        #estract the name of the category with regular expression
        cat = re.search(':(.+?);', line)
        cat = cat.group(1)
        #remove nodes that are isolated (no edges incoming, no edges outgoing)
        len_cat = len(set(list_of_art))
        categories[cat] = list(set(list_of_art).intersection(nodes))
        isolated_nod[cat] = len_cat - len(categories[cat])

#save the dictionary as a txt file
func.save_dict('categories.txt', categories)

Then, we create a dictionary *shortest* where the keys are the nodes present in our graph and the values are empty lists, we will fill them after with the shortest paths.  

In [8]:
shortest ={}
for i in nodes:
    shortest[i] = []

Now we define a Breadth First Search function, called *bfs* to compute the shortest path. This function takes as input an adjacency list (i.e. our *edges* dictionary) and a node. It will keep itarating from the starting node to all its children and so on, untill it reaches all the node than can be reachable from the starting one, during this process we take record of the depth (i.e. path): if the child node is a key of our *shortest* dictionary, we append to the value (that is a list), the depth. It is possible to see the code of our bfs in functions.py file.

We take a category in input:

In [9]:
input_ = input('Enter an input category: ')

Enter an input category: Year_of_birth_unknown


Now we compute the *bfs* for all the nodes of our input category. During the iteration the *shortest* dictionary will be updated, at the end this dictionary will have this structure: the key are all the nodes, the values are lists in which the elements are the shortest path between the key node and the node from the input category, if a key node has as value an empty list it means that node has no link with none of the nodes in the input category.   
We save the resulting *shortest* distance in a txt file (*C0_path.txt*), attached in our repository.

In [10]:
C0 = categories[input_]

for art in C0:
    shortest = func.bfs(edges, art, shortest)

#save shortest dictionary to a file
func.save_dict('C0_path.txt', shortest)

In order not to rexecute the previous code, we include here the code to reload from *C0_path.txt*, *shortest* in the right format (i.e. a dictionary):

In [11]:
#load back shortest from file as a dictionary

shortest = func.load_saved_dict('C0_path.txt')
shortest = {int(k):v for k,v in shortest.items()} #convert key into integers

Finally the comparison between categories: for every article of one category we retrive the corresponding value in *shortest* dictionary, i.e. the list with all the path between that article and all the article in the input category (C0), we append all those list in a bigger one (list for the whole category), called *distances*, and we calculate the median over it.  

Our analysis, untill now, has considered the nodes that are connected, to take into account also the nodes that are not conncted (infinite distance) with with the article of the iteration, we use this strategy: we compute the product between the lengths of input category and the one of the other category that is in the iteration, in this way we know how long should be our list of shortest paths, if this list (we called it *distance*) is smaller than that length, we calculate the difference and append as many infinite values as the value of the difference to the shortest path list (i.e. the one named *distances*). In addition, we consider how many isolated node are in the category of interest, retrieving them from *isolated_nod* dictionary. Since these nodes have no edges incoming/outgoing, surely they are not connected with any node of the imput category, therefore we add to the shortest path list as many inf value as the number of isolated nodes.  
Finally, we compute the median over that list. Then recording the median value, we rank the category.  
We decide to order the categories, fist taking into account the value of median, and second (since several categories has the same median) by the ratio of n. of infinite values over the total number of elements in its shortest path list. We opted for this method of ranking, beacuse we belive that between two categories with the same median, the one with more infinite values, is as consequence less connected to the input category.

We store the result in a form of a tuple in which the first element is the name of the category and the second is the result of the median, then we append the tuple to *vector* list.  

NOTE: all this code is inside the definition of function *distances_vector* in functions.py file.

In [13]:
vector = func.distances_vector(categories, shortest, input_, C0, isolated_nod)

We save *vector* list inside a txt file, so we can just use the file instead of rexecute the code above, and load it back as a list.

In [14]:
#save vector list to a file

func.save_list('unsort_vec.txt', vector)

In [15]:
#load vecotor from the file as a list

vector1 = func.load_saved_list('unsort_vec.txt')

In conclusion, we sort the *vector1* list, according to the value of the median, and print it out, keeping, as requested, just the names of the categories.

In [16]:
vector = sorted(vector, key=lambda x: (x[1], x[2]))
block_ranking = []
for i in vector:
    block_ranking.append(i[0]) #block ranking with sorted categories without showing median values

vector

[('Year_of_birth_unknown', -1, 0),
 ('American_film_actors', 6, 0.3015),
 ('English-language_films', 6.0, 0.3095),
 ('Members_of_the_United_Kingdom_Parliament_for_English_constituencies',
  7,
  0.2947),
 ('Indian_films', 7, 0.301),
 ('American_films', 7, 0.3022),
 ('English_television_actors', 7, 0.3318),
 ('American_television_actors', 7.0, 0.3332),
 ('British_films', 7, 0.3501),
 ('Black-and-white_films', 7, 0.3506),
 ('American_Jews', 7, 0.4044),
 ('Article_Feedback_Pilot', 7, 0.4314),
 ('Rivers_of_Romania', 8.0, 0.3377),
 ('English-language_albums', 8, 0.4273),
 ('People_from_New_York_City', 8.0, 0.4543),
 ('Debut_albums', 9.0, 0.4448),
 ('Windows_games', inf, 0.5012),
 ('Fellows_of_the_Royal_Society', inf, 0.5092),
 ('Place_of_birth_missing_(living_people)', inf, 0.5172),
 ('Living_people', inf, 0.5258),
 ('The_Football_League_players', inf, 0.5326),
 ('Harvard_University_alumni', inf, 0.5424),
 ('American_military_personnel_of_World_War_II', inf, 0.5458),
 ('English_footballers'

In [18]:
func.save_list('block_ranking.txt', block_ranking)

In [19]:
#load vector from the file as a list

block_ranking = func.load_saved_list('block_ranking.txt')
block_ranking

['Year_of_birth_unknown',
 'American_film_actors',
 'English-language_films',
 'Members_of_the_United_Kingdom_Parliament_for_English_constituencies',
 'Indian_films',
 'American_films',
 'English_television_actors',
 'American_television_actors',
 'British_films',
 'Black-and-white_films',
 'American_Jews',
 'Article_Feedback_Pilot',
 'Rivers_of_Romania',
 'English-language_albums',
 'People_from_New_York_City',
 'Debut_albums',
 'Windows_games',
 'Fellows_of_the_Royal_Society',
 'Place_of_birth_missing_(living_people)',
 'Living_people',
 'The_Football_League_players',
 'Harvard_University_alumni',
 'American_military_personnel_of_World_War_II',
 'English_footballers',
 'Year_of_birth_missing_(living_people)',
 'Major_League_Baseball_pitchers',
 'English_cricketers',
 'Association_football_forwards',
 'Association_football_defenders',
 'Association_football_midfielders',
 'Association_football_goalkeepers',
 'Year_of_birth_missing',
 'Asteroids_named_for_people',
 'Year_of_death_missi

### Ranking of articles

Once we obtained the block_ranking vector of the categories, we want to rank the articles within each category. 

In order to do so, we need to prepare our data. First, according to the order given by the block ranking, we create a list of lists, named *cats*, in which every list is the articles(i.e. nodes) that belongs to a category: e.g. *cats[0]* is a list of all article that are in the first category in the block ranking (i.e. the input category).  

Further, in the assignment it is written that in the case an article belongs to more than one category, *"if the article belongs to the input category, it belongs to that one. Otherwise, the category of the article will correspond, among the categories it belongs to, to the closest to the input category"*. To allocate correctly the articles that appears in different categories, we take all the articles that are in a category and check if they have appeard in previous categories (`confr = confr.union(cats[i-1])`) if so, we remove the article from the current category (`cats[i] = list(set(cats[i])-confr`).

In [20]:
#list of articles for each category
cats = []
for name in block_ranking:
    cats.append(categories[name])
    
#allocate articles in right category
for i in range(1, len(cats)):
    if i == 1:
        confr = set(cats[0])
    else:
        confr = confr.union(cats[i-1])
    cats[i] = list(set(cats[i])-confr)

To find the subgraph induced by a category, we used the library *networkx*, but before comupting subgraph, we need to create our graph (actually a DiGraph, directed graph) in networkx: we build an empty directed graph, then we fill with the set of all our articles, named *nodes* (created in RQ1),  and then with edges. To make the edges readable to networkx we had to present them as a list of tuple, where the first element is the start node, and the second is the end node, for this reason we open again *wiki-topcats-reduced.txt*, and create a list of tuples, as explained.

In [21]:
#create a container of edges as a list of tuples
f = open('wiki-topcats-reduced.txt')
edges_1 = []
for line in f:
    (key, val) = line.split()
    tup = (int(key),int(val))
    edges_1.append(tup)

#create graph, fill it with nodes and edges
G = nx.DiGraph()
G.add_nodes_from(nodes)
G.add_edges_from(edges_1)

Thinking about the future, we know that we are not going to show up a rank with the numbers corresponding to each article, instead we want to make the visualistaion easy to understand, so we want to print a rank of titles of the articles. With this aim, we decide to create the dictionary *page_names*, where the keys are the number corresponding to each article, and the values their title.  

In [22]:
f = open('wiki-topcats-page-names.txt')
page_names = {}

for line in f:
    (key, val) = line.split(' ', 1)
    page_names[int(key)] = val.strip('\n')

**Step 1**  
We compute the subgraph induced by C0: create a subgraph with the nodes of C0, for every node we keep record of the its degree with `sub.in_degree(nod)`, putting the node as a key and its degree as a value of c dictionary.  
We also started to built our *score article* in the form of a dictionary in which the key is the name of the category, and the value is a list of article names (thanks to *page_names* dictionary created before) ranked by degree values.

In [23]:
score_art = defaultdict(list)
c = {}
sub = nx.subgraph(G,cats[0])
key = block_ranking[0]
for nod in cats[0]:
    c[nod] = sub.in_degree(nod)
    name = page_names[nod] 
    score_art[key].append((name, c[nod])) 
score_art[key].sort(key=lambda x: x[1], reverse = True)

**Step 2-3**  
We now repeat the same operations did in the previous step for the other categories, the only difference is that this time, we have to compute the degree taking into account the fact that if an edges come from a node for which we already know the degree, the edges will be weighted with that degree.  

We iterate through category, and then through all the nodes inside that category. At the beginnig, we assign to the node the degree given by `sub.in_degree(nod)`, however this line of code consider only the edges incoming from nodes in the same subgraph (i.e. category). To take into account edges incomig from other nodes, we calculate the set difference between *G.in_edges(nod)* and *sub.in_edges(nod)*, i.e. the difference between the edges coming from the whole graph versus the edges coming from inside the subgraph, this set difference returns, between the results, the starting nodes from the other categories. From the homework assignment, we know that we have to consider the degree of those *external* nodes only if their categories have already been processed, we check that with `if item[0] in c.keys()`, if that the case, we add to the our node degree also the degree of those starting nodes (`c[nod] += c[item[0]]`).  

Then as we did in step one, we save the node degree in c dictionary, and update the *score_art* list with ranked articles.

In [24]:
for n in range(1, len(cats)):
    key = block_ranking[n]
    sub = nx.subgraph(G,cats[n])
    for nod in cats[n]:
        c[nod] = sub.in_degree(nod)
        diff = set(G.in_edges(nod))-set(sub.in_edges(nod))
        if len(diff) > 0:
            for item in diff:
                if item[0] in c.keys():
                    c[nod] += c[item[0]]                    
        name = page_names[nod]
        score_art[key].append((name, c[nod]))
    score_art[key].sort(key=lambda x: x[1], reverse = True)

In [25]:
#convert from defaultdict to dict
score_art1 = dict(score_art)

#save score_art dictionary to a file
func.save_dict('score_art.txt', score_art1)

In [26]:
#load score_art dictionary from file back to dict type
score_art1 = func.load_saved_dict('score_art.txt')

For visualisation purposes we show our score articles as a dataframe.  
Note: None values appears because same categories has more articles than others.

In [27]:
df_score = pd.DataFrame.from_dict(score_art1, orient='index')
df_score

,0,1,2,3,4,5,6,7,8,9,...,323996,323997,323998,323999,324000,324001,324002,324003,324004,324005
Year_of_birth_unknown,"[Diogenes Lartius, 21]","[Pausanias (geographer), 12]","[L Bu, 10]","[Dong Zhuo, 10]","[Theocritus, 10]","[Yuan Shu, 9]","[Stobaeus, 7]","[Penda of Mercia, 7]","[Symeon of Durham, 7]","[Zhang Fei, 6]",...,None,None,None,None,None,None,None,None,None,None
American_film_actors,"[John Wayne, 278]","[Clint Eastwood, 202]","[Elvis Presley, 198]","[Woody Allen, 196]","[Frank Sinatra, 192]","[Steven Spielberg, 190]","[Marilyn Monroe, 146]","[Henry Fonda, 139]","[Bette Davis, 138]","[Lucille Ball, 138]",...,None,None,None,None,None,None,None,None,None,None
English-language_films,"[Gone with the Wind (film), 1243]","[Terror in the Aisles, 1026]","[The Searchers (film), 951]","[Apocalypse Now, 853]","[The Wizard of Oz (1939 film), 845]","[Dirty Harry, 825]","[The Godfather Part II, 823]","[The Aviator (2004 film), 767]","[The Departed, 745]","[True Grit (1969 film), 734]",...,None,None,None,None,None,None,None,None,None,None
Members_of_the_United_Kingdom_Parliament_for_English_constituencies,"[Winston Churchill, 2788]","[Tony Blair, 1741]","[Margaret Thatcher, 564]","[Benjamin Disraeli, 549]","[Neville Chamberlain, 395]","[John Bright, 365]","[Michael McGuire, 348]","[David Cameron, 336]","[Arthur Wellesley, 1st Duke of Wellington, 318]","[Glenda Jackson, 313]",...,None,None,None,None,None,None,None,None,None,None
Indian_films,"[Dhamaal, 682]","[Three Kings (2011 film), 680]","[Life in a... Metro, 502]","[Chori Chori, 449]","[Rafoo Chakkar, 449]","[Ghulam, 444]","[Disco Dancer, 413]","[Akele Hum Akele Tum, 375]","[Musafir (2004 film), 344]","[Avvai Shanmughi, 330]",...,None,None,None,None,None,None,None,None,None,None
American_films,"[Bonnie and Clyde (film), 4233]","[Easy Rider, 3260]","[The Conversation, 2887]","[Shrek 2, 2797]","[The Shootist, 2718]","[Friday the 13th (1980 film), 2294]","[The Rains Came, 2250]","[The Big Sleep (1946 film), 2080]","[Our Gang, 1997]","[Twilight (2008 film), 1888]",...,None,None,None,None,None,None,None,None,None,None
English_television_actors,"[Bob Hope, 13106]","[Laurence Olivier, 9614]","[Michael Caine, 9150]","[David Niven, 8470]","[Peter Sellers, 8226]","[Peter Lawford, 7733]","[Jude Law, 6745]","[Albert Finney, 6684]","[Ronald Colman, 6591]","[Reginald Owen, 6523]",...,None,None,None,None,None,None,None,None,None,None
American_television_actors,"[Will Smith, 14323]","[Dan Rowan, 7889]","[James Earl Jones, 7623]","[Hattie McDaniel, 6663]","[David Hasselhoff, 6620]","[Connie Booth, 6439]","[Magda Gabor, 5688]","[Wesley Snipes, 5341]","[Linda Evans, 5182]","[Alfred Drake, 4999]",...,None,None,None,None,None,None,None,None,None,None
British_films,"[Casino Royale (1967 film), 30584]","[Battle of Britain (film), 21004]","[Bullseye!, 20921]","[Henry V (1944 film), 18902]","[Maurice (film), 18284]","[Ella Enchanted (film), 16799]","[War Requiem (film), 16690]","[The Man Who Never Was, 16501]","[The Fixer (film), 15961]","[Tube Tales, 15619]",...,None,None,None,None,None,None,None,None,None,None
Black-and-white_films,"[In Which We Serve, 34492]","[The Cabinet of Dr. Caligari, 31582]","[The Naked Truth (1957 film), 22070]","[A Hard Day's Night (film), 20911]","[The Ghost Breakers, 20402]","[Cluny Brown, 17731]","[Arms and the Man, 16528]","[Road to Rio, 14753]","[Kind Hearts and Coronets, 14581]","[Julia Misbehaves, 14419]",...,None,None,None,None,None,None,None,None,None,None


In conclusion we present our results in following the requests of the assignment:

In [28]:
final_rank = []
for i in score_art1.values():
    for j in i:
        final_rank.append(j[0])

In [29]:
#save sorted_vector_withval list to a file

func.save_list('final_rank.txt', final_rank)

In [30]:
#load final_rank from file back to a list type

final_rank = func.load_saved_list('final_rank.txt')

Take a look at the top of our list of ranked articles:

In [31]:
#take a look to a chunck of our final rank 
final_rank[:30]

['Diogenes Lartius',
 'Pausanias (geographer)',
 'L Bu',
 'Dong Zhuo',
 'Theocritus',
 'Yuan Shu',
 'Stobaeus',
 'Penda of Mercia',
 'Symeon of Durham',
 'Zhang Fei',
 'Stephen Dingate',
 'Edwin, Earl of Mercia',
 'Stigand',
 'Valerius Maximus',
 'Pope Alexander II',
 'Pope Nicholas III',
 'Sweyn II of Denmark',
 'Pope Nicholas II',
 'Tancred of Hauteville',
 'Ivan Alexander of Bulgaria',
 'Nefertiti',
 'Horemheb',
 'Sextus Empiricus',
 'Amasis II',
 'Devapala',
 'Pope John VIII',
 'Sir William Osborne, 8th Baronet',
 'Joseph N. Langan',
 'Marcus Licinius Crassus Dives (consul 14 BC)',
 'Florus']